In [ ]:
# boring technical stuff
import os, sys
import random
import time
import imutils
import math
import numpy as np
import pandas as pd
import skimage.io
import matplotlib
import matplotlib.pyplot as plt
import cv2
from matplotlib.patches import Polygon

### List of trackers to run through for each video file

In [ ]:
trackers = {'boosting': cv2.TrackerBoosting_create(),
                'mil': cv2.TrackerMIL_create(),
                'kcf': cv2.TrackerKCF_create(),
                'tld': cv2.TrackerTLD_create(),
                'medianflow': cv2.TrackerMedianFlow_create(),
                'goturn': cv2.TrackerGOTURN_create(),
                'mosse': cv2.TrackerMOSSE_create(),
                'csrt': cv2.TrackerCSRT_create()}

In [ ]:
clips = {'Bike02': '/Users/kierandonnelly/thesis/raw_clips/Bike02.mp4',
        'Bike03': '/Users/kierandonnelly/thesis/raw_clips/Bike03.mp4',
        'Bike05': '/Users/kierandonnelly/thesis/raw_clips/Bike05.mp4',
        'Bike07': '/Users/kierandonnelly/thesis/raw_clips/Bike07.mp4',
        'Bike08': '/Users/kierandonnelly/thesis/raw_clips/Bike08.mp4',
        'Bike09': '/Users/kierandonnelly/thesis/raw_clips/Bike09.mp4',
        'Ski01': '/Users/kierandonnelly/thesis/raw_clips/Ski01.mp4',
        'Ski02': '/Users/kierandonnelly/thesis/raw_clips/Ski02.mp4',
        'Ski03': '/Users/kierandonnelly/thesis/raw_clips/Ski03.mp4',
        'Snowboard01': '/Users/kierandonnelly/thesis/raw_clips/Snowboard01.mp4',
        'Sup01': '/Users/kierandonnelly/thesis/raw_clips/Sup01.mp4',
        'Surf01': '/Users/kierandonnelly/thesis/raw_clips/Surf01.mp4',
        'Wake01': '/Users/kierandonnelly/thesis/raw_clips/Wake01.mp4'}

### Iterate over videos in directory and run tracker

In [ ]:
def run_track_test(input1, input2):
    return(print(input1, input2))

In [ ]:
frame_list = []

# iterate over clips
for clip, clip_loc in clips.items(): 
    
    # retrieve frame data for clip
    frames = pd.read_csv(str('./frame_data/'+str(filename)+'.csv'))
    
    # iterate over tracker dictionary
    for tracker, tracker_inst in trackers.items(): 

        # call tracking function
        run_track_test(tracker_inst, clip_loc, frames)

### Import ground truth annotation data for each video

In [ ]:
videos_df = pd.read_csv("./frame_data/Bike05.csv"); videos_df.head()

In [17]:
frames = pd.read_csv(str('./frame_data/Bike03.csv'))
frame_count = 3

In [23]:
frames.head()

,frame,x1,y1,w,h
0,1.0,0.006649,0.178487,0.067819,0.117021
1,2.0,0.018617,0.186761,0.067154,0.112293
2,3.0,0.034574,0.193853,0.064495,0.106383
3,4.0,0.043883,0.198582,0.068484,0.115839
4,5.0,0.057181,0.210402,0.068484,0.105201


In [24]:
init_bb = frames.loc[frames['frame'] == frame_count].squeeze()[1:]; init_bb

x1    0.034574
y1    0.193853
w     0.064495
h     0.106383
Name: 2, dtype: float64

### Function to implement tracking on one video sequence

In [ ]:
# this runs once per clip per tracker
# lists declared and modified here pertain to one clip
def run_track(tracker_inst, video_file, frames_df):
    '''args: '''
    
    # initialise vars
    frame_count = frames_df['frame'].min() # first frame containing g.t. object
    miss_count, fp_count = 0, 0 # counters for different type of events
    iou_list = [] # keep track of ious for each frame
    
    # define bb to initialise tracker with (x1, y1, w, h)
    init_bb = frames_df.loc[frames_df['frame'] == frame_count].squeeze()[1:]
    
    # Initialize the video stream and pointer to output video file
    vs = cv2.VideoCapture(video_file)
    
    # run until no more frames
    while True:
        # read the next frame from the file
        grabbed, frame = vs.read()

        # if no frame grabbed, reached end of clip
        if not grabbed:
            print ("Not grabbed!")
            break

        # will resize the frame for faster processing in our NN if applicable, and save the resolution
        if resize is not 0:
            frame = imutils.resize(frame, int(resize))

        # start timer for fps metric
        detectorTime = time.time()
        
        # initialize the writer
        if writer is None:
            fourcc = cv2.VideoWriter_fourcc(*"MP4V")
            fps = vs.get(5)
            print("FPS = " + str(fps))
            writer = cv2.VideoWriter("output/" + args["inputVideo"] + "_" + "_" + args["tracker"] + ".mp4", fourcc, fps, (frame.shape[1], frame.shape[0]), True)
        
        if tracker is None:
            # create tracker object
            tracker = tracker_inst
            
            # initialise with some bounding box
            tracker.init(frame, (init_bb[0], init_bb[1], init_bb[2], init_bb[3]))
        else: 
            # tracker is already initialised
            # update the tracker and grab the position of the tracked object
            tracking, bbox = tracker.update(frame)
            
            # check if tracking, then draw bounding box
            if tracking:
                # Tracking success
                p1 = (int(bbox[0]), int(bbox[1]))
                p2 = (int(bbox[0] + bbox[2]), int(bbox[1] + bbox[3]))
                cv2.rectangle(frame, p1, p2, (255, 0, 0), 2, 1)
                cv2.putText(frame, text, (int(bbox[0]), int(bbox[1]) - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.45, (255, 0, 0), 2)
                trackingFailed = False
            else:
                # Tracking failure
                cv2.putText(frame, "Tracking failure detected", (100,80), cv2.FONT_HERSHEY_SIMPLEX, 0.75,(0,0,255),2)
                trackingFailed = True

            # logging purposes
            if not trackingFailed:
                print("[INFO] Tracker running at " + str(1/(time.time() - trackerTime)) + " fps")
                trackingFPS += 1/(time.time() - trackerTime)
                trackingFrames += 1
                
        # increment frame counter
        frame_count += 1
        
        # write the frame to disk
        if writer is not None:
            writer.write(frame)
        
        # TODO calculate metrics

## Evaluation Metrics
This section defines functions to compute the various evaluation metrics detailed in Chapter 3 in the report. The metrics to compute are as follows:
- Recall: Correctly matched detections as proportion of total ground truth objects (of a sequence).
- Precision/N-SODA: Correctly matched detections as a proportion of total detections (of a sequence).
- FAF: Number of false alarms (incorrect detections) per frame averaged over a sequence.
- SODP: Average overlap between true positives and ground truth.
- SOTA: Combines false negatives and false positives.
- SOTP: Overlap between the tracker output and the ground truth averaged over the matches.
- TDE: Distance beetween the ground-truth annotation and the tracking result.
- MT: The ground-truth trajectory is covered by the tracker output for more than 80% of its length.
- ML: The ground-truth trajectory is covered by the tracker output for less than 20% of its length.
- PT: The ground-truth trajectory is covered by the tracker output for between 20% and 80% of its length.
- FM: Number of times that a ground-truth trajectory is interrupted in the tracking result, normalised over sequence.
- RS: Ratio of tracks which correctly recover from short term occlusion.
- RL: Ratio of tracks which correctly recover from long term occlusion.

### Terminology
- **Correctly-matched**: aka True Positive. Overlap between ground truth and tracker output is non-zero.
- **Detection**: Tracker has some output for the frame.
- **Miss**: aka False Negative. No output from tracker.
- **False Positive**: Overlap between tracker output and ground truth is zero.
- **Distance**: Euclidean distance between centroids of tracker output and ground truth.
- **Covered**: Overlap is non-zero.

### Elements for whole sequence

In [ ]:
# non-zero overlaps/correctly matched detections
# cmd = 

# total ground truth objs in sequence/number of frames for a clip in the annotated data
# in annotation table, groupby clip and then count rows for that clip
# gto = 

# total detections made/tracker output non-zero
# count tracking attempts made for a clip
# td = 

# false positives/overlap of zero
# fp = 

### Elements for single frame

#### Mapped Overlap Ratio (MOR)

In [ ]:
# mapped overlap ratio
def metric_mor(boxA, boxB):
    # (x1,y1,x1 + w,y1 + h) -> (x1, y1, x2, y2)
    boxA = [boxA[0], boxA[1], boxA[0] + boxA[2], boxA[1] + boxA[3]]
    boxB = [boxB[0], boxB[1], boxB[0] + boxB[2], boxB[1] + boxB[3]]
    
    # determine the (x, y)-coordinates of the intersection rectangle
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])

    # compute the area of intersection rectangle
    interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)

    # compute the area of both the prediction and ground-truth
    # rectangles
    boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
    boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)

    # compute the intersection over union by taking the intersection
    # area and dividing it by the sum of prediction + ground-truth
    # areas - the interesection area
    iou = interArea / float(boxAArea + boxBArea - interArea)

    # return the intersection over union value
    return iou

#### Recall

In [ ]:
def metric_recall(cmd, gto):
    return cmd/gto

#### Precision

In [ ]:
def metric_precision(cmd, td):
    return cmd/td

#### False Alarms per Frame (FAF)

In [ ]:
def metric_faf(fp, gto):
    return fp/gto

#### Single Object Detection Precision

In [ ]:
def metric_sodp(sum_mor, n_frames):
    # sum_mop = sum the mop() for each frame
    # n_frames = len(sequence)
    return sum_mor/n_frames

### Testing Area